In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
import h5py

import csv

import glob2

Autosaving every 180 seconds


In [11]:
# 
class Visualize():
    
    def __init__(self, 
                 tracker):
              
        # 
        self.tracker = tracker
    
#     #    
#     def get_tracks_from_h5(self):
        
#         # 
#         fname_out = self.fname_h5[:-3]+'.npy'
        
#         # 
#         if os.path.exists(fname_out)==False:

#             # convert data from h5 to npy
#             hf = h5py.File(self.fname_h5, 'r')
#             keys = hf.keys()
#             group2 = hf.get('tracks')
#             tracks = []

#             for k in range(len(group2)):
#                 tracks.append(group2[k])

#             tracks = np.array(tracks).transpose(3,0,2,1)
#             np.save(self.fname_h5[:-3]+'.npy',tracks)
#             print ("[n_frames, n_animals, n-features, xy,]", tracks.shape)

#             self.tracks = tracks
#         else:
#             self.tracks = np.load(fname_out)
            
        
        
    def make_video_skeleton(self,
                            fps = 25,
                           start=None,
                           end=None):

        from tqdm import trange

        colors = [
            (0,0,255),
            (255,0,0),
            (255,255,0),
            (0,128,0)
            ]

        # 
        names = ['female','male','pup1','pup2']
        clrs = ['blue','red','cyan','green']
      
        if start is None:
            start =0
        if end is None:
            end = self.tracks.shape[0]
            
        # load and videos    
        fname_out = (self.fname_video[:-4]+'_'+str(start)+"_"+str(end)+'.mp4')

        # video settings
        size_vid = np.array([1280,1024])
        dot_size = 4
        thickness=-1

        # load original vid
        original_vid = cv2.VideoCapture(self.fname_video)
        fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
        video_out = cv2.VideoWriter(fname_out,
                                    fourcc, 
                                    fps, 
                                    (size_vid[0],size_vid[1]), 
                                    True)
        
        original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

        font = cv2.FONT_HERSHEY_PLAIN


        # loop over frames
        for n in trange(start, end, 1):
        #for n in trange(start, 100, 1):
            ret, frame = original_vid.read()

            cv2.putText(frame, str(n), (50, 50), font, 5, (255, 255, 0), 5)

            for i in range(self.tracks.shape[1]):
                color = colors[i] #(255, 0, 0)

                for f in range(self.tracks.shape[2]):
                    x = self.tracks[n,i,f,0]
                    y = self.tracks[n,i,f,1]

                    if np.isnan(x) or np.isnan(y):
                        continue

                    x = int(x)
                    y = int(y)

                    # plot cicrl
                    center_coordinates = (x,y)
                    radius = dot_size
                    frame = cv2.circle(frame, center_coordinates, radius, color, thickness)



                    # plot line
                    if f<(self.tracks.shape[2]-1):
                        if np.isnan(self.tracks[n,i,f+1,0])==False:
                            start_point = (int(self.tracks[n,i,f+1,0]),
                                           int(self.tracks[n,i,f+1,1])) 

                            # End coordinate, here (250, 250) 
                            # represents the bottom right corner of image 
                            end_point = center_coordinates

                            # Line thickness of 9 px 
                            thickness = 3

                            # Using cv2.line() method 
                            # Draw a diagonal green line with thickness of 9 px 
                            frame = cv2.line(frame, 
                                             start_point, 
                                             end_point, 
                                             color, 
                                             thickness)

    #                 frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
    #                     matplotlib.colors.to_rgb(clrs[i]))*255.).astype('uint8')

            video_out.write(frame)

        video_out.release()
        original_vid.release()


    def make_video_centroid(self,
                            tracks,
                            fname_video,
                            start,
                            end,
                            fps):

        from tqdm import trange

        colors = [
            (0,0,255),
            (255,0,0),
            (255,255,0),
            (0,128,0)
            ]

        # 
        names = ['female','male','pup1','pup2']
        clrs = ['blue','red','cyan','green']

        if start is None:
            start =0
        if end is None:
            end = tracks.shape[0]
        
        # load and videos    
        video_name = fname_video
        fname_out = (video_name[:-4]+'_'+str(start)+"_"+str(end)+'_centroid.mp4')
        if False: #os.path.exists(fname_out):
            print ("fname: exists", fname_out)
            return

        # video settings
        size_vid = np.array([1280,1024])
        dot_size = 12
        thickness = -1

        # load original vid
        original_vid = cv2.VideoCapture(video_name)
        fourcc = cv2.VideoWriter_fourcc('M','P','E','G')

        video_out = cv2.VideoWriter(fname_out,fourcc, fps, (size_vid[0],size_vid[1]), True)
        original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

        font = cv2.FONT_HERSHEY_PLAIN


        # loop over frames
        histories = np.zeros((tracks.shape[1],
                              5,2),'float32')+np.nan
        print ("Histories: ", histories.shape)
        for n in trange(start, end, 1):
        #for n in trange(start, 100, 1):
            ret, frame = original_vid.read()

            cv2.putText(frame, str(n), (50, 50), font, 5, (255, 255, 0), 5)

            #
            for i in range(tracks.shape[1]):
                color = colors[i] #(255, 0, 0)

                #for f in range(tracks.shape[2]):
                x = tracks[n,i,0]
                y = tracks[n,i,1]

                try:
                    histories[i,1:]=histories[i,:4]
                    histories[i,0,0]=x
                    histories[i,0,1]=y
                except:
                    histories[i,1:]=histories[i,:4]
                    histories[i,0,0]=np.nan
                    histories[i,0,1]=np.nan

                # plot centroids and history lines
                for h in range(5):
                    center_coordinates = (histories[i,h,0], histories[i,h,1])
                    #if i==0:

                    # check centre points
                    idx = np.where(np.isnan(center_coordinates)==True)[0]
                    if idx.shape[0]==0:
                        radius = int(dot_size*(1-h/5/2))
                        frame = cv2.circle(frame, 
                                           center_coordinates, 
                                           radius, 
                                           color, 
                                           -1)
                    else:
                        continue

                    # draw lines
                    if h<4:
                        start_point = center_coordinates
                        end_point = (histories[i,h+1,0], 
                                         histories[i,h+1,1])

                        idx = np.where(np.isnan(end_point)==True)[0]
                        if idx.shape[0]==0:
                            # Line thickness of 9 px 
                            thickness = 3

                            # Using cv2.line() method 
                            # Draw a diagonal green line with thickness of 9 px 
                            frame = cv2.line(frame, 
                                             start_point, 
                                             end_point, 
                                             color, 
                                             thickness)



            video_out.write(frame)

        video_out.release()
        original_vid.release()
   
#     def show_tracks(self, start, end):
        
#         #tracks = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.npy')
#         #print (tracks.shape)
#         clrs = ['red','blue','cyan','green']
#         names = ['female','male','pup1','pup2']
#         #t=np.arange(tracks.shape[0])#/25.
#         print (self.tracks.shape)
#         t= np.arange(start,end,1)
#         #for k in range(start, end,1):
#         for k in range(self.tracks.shape[1]):
#             temp = self.tracks[start:end,k]
#             temp = np.nanmean(temp,axis=1)

#             # plot centre of animals
#             temp = np.sum(temp,axis=1)
#             plt.plot(t,temp,c=clrs[k],label=names[k])

#             print ('')
#         plt.legend(fontsize=10)
#         plt.xlabel("Frames",fontsize=20)
#         plt.show()


    def show_track_centers(self, 
                           tracks, 
                           start, 
                           end):
        
        clrs = ['red','blue','cyan','green']
        names = ['female','male','pup1','pup2']

        t= np.arange(start,end,1)

        for k in range(tracks.shape[1]):
            temp = tracks[start:end,k].sum(1)
            
            #plt.plot(t,temp,c=clrs[k],label=names[k])
            plt.scatter(t,temp,c=clrs[k],label=names[k])
            
            print ('')
        plt.legend(fontsize=10)
        plt.xlabel("Frames",fontsize=20)
        plt.show()

# 
class Tracker():
    
    # 
    def __init__(self, 
                 tracks):
        
        #
        self.tracks = tracks.copy()
    
        # 
        self.tracks_corrected = self.tracks.copy()
        
        #
        self.tracks_corrected_centers = np.nanmean(
                                                self.tracks_corrected,
                                                axis=2)
  
      
    

In [14]:
# Simplified motion predictor without variance/bayes rule updates
# as our measurements are NOT noisy (a simplified Kalman filter)
import scipy
import scipy.spatial
from scipy.optimize import linear_sum_assignment
from tqdm import trange

class Predict():
    
    def __init__(self, tracks, verbose):
        
        #
        self.tracks = tracks
        print ("input into Predict: ", self.tracks.shape)
        
        # 
        self.tracks_fixed = np.zeros(self.tracks.shape,'float32')
                
        #
        self.verbose=verbose
    
    def get_positions(self):     

        '''     ###############################################
                ###### GET ANIMAL POSITIONS AT TIME T #########
                ###############################################
                # get x,y positions of each animal 
                # for most recent 3 values
                # eventually, add uncertainty based on how far back we go; BUT COMPLICATED
        '''
        
        # Problem: we don't always have data in sequence so need to work on interpolating also
        # make array [n_animals, n_time_points, 2]
        n_time_points = 3
        self.pos= np.zeros((self.tracks.shape[1], n_time_points, 2), 'float32')
        for a in range(self.tracks.shape[1]):
            
            # TO IMPLEMENT LATER
            # loop back in time to find nearest non zero vals;
            #for t1 in np.arange(t,0,-1):
            #    if np.isnan(self.tracks[t1,a,0])==False:
            #        self.pos[a]= self.tracks[t1-3:t1,a]
            #        break        
        
            self.pos[a]=self.tracks[self.time-3:self.time,a]
    
            # temporarily replace read positions that are nans with most recent non-nan val
            idx = np.where(np.isnan(self.pos[a]))[0]
            for id_ in idx:
                idx2 = np.where(np.isnan(self.tracks_fixed[:self.time,a])==False)[0]
                self.pos[a] = self.tracks_fixed[idx2[-1],a]
#             # 
#             idx = np.where(np.isnan(self.pos[a]))[0]
#             if idx.shape[0]>0:
#                 print ('found nan data', a, self.pos[a])
#                 self.pos[a,idx]==self.pos[a,2]
        
        if self.verbose:
            print ("self.positions; ", self.pos)
                        
    
    def compute_vel_acceleration(self):
        '''###############################################
        ###### COMPUTE VELOCITY AND ACCELERATION ######
        ###############################################
        # compute velocity from 2 values
        # compute acceleration from 3 values
        '''

        self.vel = np.zeros((self.tracks.shape[1], 2), 'float32')
        self.acc = np.zeros((self.tracks.shape[1], 2), 'float32')

        # turn off predictive dynamics, seems to be mostly problematic
        if self.dynamics_off_flag==False:

            for a in range(self.tracks.shape[1]):
                vel = self.pos[a,2]-self.pos[a,1]
                acc = (self.pos[a,2]-self.pos[a,1])- \
                      (self.pos[a,1]-self.pos[a,0])
                self.vel[a]=vel
                self.acc[a]=acc

            # set all vel and acc nans to zeros
            idx = np.where(np.isnan(self.vel))
            self.vel[idx]=0
            idx = np.where(np.isnan(self.acc))
            self.acc[idx]=0            
        
        # use dampening on acceleration vector - animals can't reach inifite velocities
        #self.acc[:]=0
        
        if self.verbose:
            print ("vel; ", self.vel)        
            print ("accel; ", self.acc)        
        
        
    def compute_position(self, 
                         p0,
                         vel0,
                         acc0):

        # for now assume dt==1 always
        # return p0 + vel0*dt + 0.5*acc0*(dt**2)

        return p0 + vel0 + 0.5*acc0


    def predict_location(self):
        
        '''###############################################
        ###### PREDICT LOCATION OF EACH ANIMAL ########
        ###############################################
        # predict location at the next time step 
        # may also wish to run prediction backwards in time as well
        '''
        
        self.pos_pred = np.zeros((self.tracks.shape[1], 2), 'float32')
        
        for a in range(self.tracks.shape[1]):
            self.pos_pred[a]= self.compute_position(self.pos[a,2],
                                                self.vel[a],
                                                self.acc[a])
        if self.verbose:
            print ("self.pos_predicted: ", self.pos_pred)
    
    
    def compute_distances(self):

        '''#######################################################
        ### DISTANCE MATRIX BETWEEN OBSERVATIONS/PRECITIONS ###
        #######################################################
        # compute distance matrix
        '''
        
        # read locations at time t+1
        self.pos_read = self.tracks[self.time+1]
        
        if self.verbose:
            print ("self.pos_read at t+1:", self.pos_read)
            
        # compute pairwise dist between predicted and read
        cost = scipy.spatial.distance.cdist(self.pos_pred, self.pos_read)
        if self.verbose:
            print ("Cost: ", cost)
    
        # NEED TO SPEED THIS UP
        idx = np.where(np.isnan(cost))
        cost[idx]=1E4
        
        # 
        _, assignment = linear_sum_assignment(cost)
        self.assignment = assignment

        #
        if self.verbose:
            print ("assignemnt: ", assignment)
            print ("costs: ", cost[_,assignment])
    
    
    def update_positions(self):
        
        ''' ###############################################
            ########### HUNGARIAN ASSIGNMENT ##############
            ###############################################
        #    assign observed data to each animal
        
            if nan, just use previous
        '''
        
        # 
        for a in range(self.assignment.shape[0]):
            
            # check if updated position is a nan and replace it with most recent non-nan
            #print (self.pos_read[self.assignment[a]])
            temp1 = self.pos_read[self.assignment[a]]
            if np.isnan(temp1[0]):
                idx2 = np.where(np.isnan(self.tracks_fixed[:self.time,a])==False)[0]
                temp1 = self.tracks_fixed[idx2[-1],a]
            
            # shift 1 cell over and grab new val
            #print (self.pos[a,1:3].shape, temp1.shape)
            self.pos[a] = np.vstack((self.pos[a,1:3], temp1))

            # save data
            self.tracks_fixed[self.time,a]=self.pos_read[self.assignment[a]]
        
        if self.verbose:
            print ("new hugnarian positoins: ", self.pos)
        
# 
tracks = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.npy')

# make tracker object
tracker = Tracker(tracks)

# initialize prediction 
verbose=False
p = Predict(tracker.tracks_corrected_centers,
           verbose)

start = 57800
end = 58200
#start = 400
#end = 500

# get positions at time t
p.time = start
p.dynamics_off_flag=True

# 
#p.delete_short_segs()

#
p.get_positions()
while True:
    p.compute_vel_acceleration()
    p.predict_location()
    p.compute_distances()
    p.update_positions()
    p.time+=1
    if p.time>end:
        break

<ipython-input-11-6f02edf95499>:316: RuntimeWarning: Mean of empty slice
  self.tracks_corrected_centers = np.nanmean(


input into Predict:  (89989, 4, 2)


In [16]:
# visualize initial reslut
vis = Visualize(p.tracks)

#
ax=plt.subplot(2,1,1)
vis.show_track_centers(p.tracks,
                       start,
                       end)
plt.title("Original")

# visualize post-prediction
ax=plt.subplot(2,1,2)
vis.show_track_centers(p.tracks_fixed,
                       start,
                       end)
plt.title("Fixed")
plt.show()



In [22]:
# 
fname_video = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.avi'
fps=5
vis = Visualize(p)

vis.make_video_centroid(p.tracks_fixed,
                        fname_video,
                        start,
                        end,
                        fps)
                        


  0%|          | 0/400 [00:00<?, ?it/s]<ipython-input-11-6f02edf95499>:223: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame,
<ipython-input-11-6f02edf95499>:244: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.line(frame,
  2%|▏         | 6/400 [00:00<00:06, 57.30it/s]

Histories:  (4, 5, 2)


100%|██████████| 400/400 [00:07<00:00, 53.21it/s]


In [523]:
# FUNCTION TO TRY AND MASK OUT BLOCKS; 
# MOSTLY FAILED

import cv2
from scipy import ndimage


class Video():
    
    def __init__(self, fname):
        
        self.fname = fname
        
        
    def init_vid(self, n):
        
        self.font = cv2.FONT_HERSHEY_PLAIN

        # load original vid
        self.original_vid = cv2.VideoCapture(self.fname)
        self.original_vid.set(cv2.CAP_PROP_POS_FRAMES, n)
        
    def init_save_vid(self, start, end):

        fps = 25
        size_vid = np.array([1280,1024])
        
        fname_out = self.fname[:-4]+"_"+str(start)+"_"+str(end)+'.mp4'

        fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
        self.video_out = cv2.VideoWriter(fname_out,
                                         fourcc, 
                                         fps, 
                                         (size_vid[0],size_vid[1]), 
                                         True)
        
    def save_vid(self, frame, surface, n):

        surface = (surface-np.min(surface))/(np.max(surface)-np.min(surface))*255
       
        self.surface[self.width:-self.width+1, 
                     self.width//2:-self.width//2+1,1] = surface.astype('uint8')
        
        frame = cv2.addWeighted(frame,0.0,
                                self.surface,1,0)

        cv2.putText(frame, str(n), (50, 50), self.font, 5, (255, 255, 0), 5)
            
            
        self.video_out.write(frame)

        
        
        
    def grab_frame(self):
        ret, frame = self.original_vid.read()
        
        return frame

    def show(self, frame):
        
        plt.imshow(frame)
        plt.show()
            
    
    def fit_mask(self, 
                frame):
        
        # print (frame.shape, mask.shape)
        
        
        frame_1d = frame[:,:,1]
        
        # find max for several rotations
        maxval=0
        surface = None
        for k in range(len(self.masks)):
            res = cv2.matchTemplate(frame_1d, 
                                    self.masks[k], 
                                    cv2.TM_CCOEFF)

            temp_maxval = np.amax(res)
            if temp_maxval>maxval:
                x,y = np.where(res==temp_maxval)
                maxval = temp_maxval
                surface=res
                #print ("new max found",temp_maxval, x,y)

        x=int(x+self.width)
        y=int(y+self.width)

        frame[x-self.width_mask:x+self.width_mask,
              y-self.width_mask:y+self.width_mask]=0
        
        

        return frame, x, y, surface
   

    def make_masks(self, mask):
        
        masks = []
        for k in range(-15,15,10):
            masks.append(ndimage.rotate(mask[:,:,1], k, reshape=False))
            
        self.masks=masks 
#    
fname = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_16_08_59_17_534732_compressed/2020_3_16_08_59_17_534732_compressed.avi'
vid = Video(fname)

# mask size
x = 674
y = 614


# make rotated masks based on frame 0 block location
start=0
vid.init_vid(start)
frame = vid.grab_frame()

vid.width=32
vid.width_mask=64
mask = frame[x-vid.width:x+vid.width,
             y-vid.width//2:y+vid.width//2]
  
vid.make_masks(mask)

# restart 
start = 600
end = 650
vid.init_vid(start)
frame = vid.grab_frame()
vid.surface = np.zeros(frame.shape, 'uint8')

# initilize save vid
vid.init_save_vid(start,end)

for n in trange(start,end,1):
    
    # return masked frame and new x,y locs
    frame_masked, x, y, surface = vid.fit_mask(frame)
    
    # 
    vid.save_vid(frame_masked, surface, n)
    
    # 
    frame = vid.grab_frame()

vid.video_out.release()
        


100%|██████████| 50/50 [00:07<00:00,  6.32it/s]


In [512]:
print (z.shape)
ax=plt.subplot(1,2,1)
plt.imshow(surface)


ax=plt.subplot(1,2,2)
#plt.imshow((surface/np.max(surface)*255).astype('uint8'))
print (np.min(surface))

plt.show()

(897, 1153)
0.0


In [366]:
plt.imshow(mask)
plt.show()